In [1]:
# !pip install SimpleITK      
# !pip install medpy          # Required for preprocessing
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tensorflow as tf
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler
from tensorflow import keras
from skimage.io import imsave
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from collections import defaultdict
from sklearn.model_selection import KFold
writer = SummaryWriter("runs/new")

torch.manual_seed(7)

import os
# os._exit(00)
os.chdir('drive/MyDrive/CAMUS_DATASET/WEIGHTS/Stuff/src')
from preprocessing import*
from model import*
from dataset import *
from train import *
from loss import*
os.chdir('../../')


In [ ]:
image = torch.rand((4,1,128,128))
image.to(device='cuda')
model = ResUnetPlusPlus_Path(16)
print(model(image).shape)
summary(model.cuda(), (1,128,128))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


torch.Size([4, 4, 128, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             160
       BatchNorm2d-2         [-1, 16, 128, 128]              32
              ReLU-3         [-1, 16, 128, 128]               0
            Conv2d-4         [-1, 16, 128, 128]           2,320
            Conv2d-5         [-1, 16, 128, 128]             160
 AdaptiveAvgPool2d-6             [-1, 16, 1, 1]               0
            Linear-7                    [-1, 1]              16
              ReLU-8                    [-1, 1]               0
            Linear-9                   [-1, 16]              16
          Sigmoid-10                   [-1, 16]               0
Squeeze_Excite_Block-11         [-1, 16, 128, 128]               0
      BatchNorm2d-12         [-1, 16, 128, 128]              32
             ReLU-13         [-1, 16, 128, 128]               0
       

In [3]:
train_ED_np = (np.load('ED_train_cv2.npy')).reshape((900,1,256,256))
train_ED_gt_np = (np.load('ED_train_gt_cv2.npy')).reshape((900,1,256,256)).astype(int)
train_ES_np = (np.load('ES_train_cv2.npy')).reshape(900,1,256,256)
train_ES_gt_np = (np.load('ES_train_gt_cv2.npy')).reshape((900,1,256,256)).astype(int)

train_ED_np = np.concatenate((train_ED_np, train_ES_np), axis=0)
train_ED_gt_np = np.concatenate((train_ED_gt_np, train_ES_gt_np), axis=0)

data_train, data_test, mask_train, mask_test = train_test_split(train_ED_np, train_ED_gt_np, test_size=0.20, random_state=42)
batch_size = 16
train_dataset = SEMDataTrain(data_train, mask_train)
val_dataset = SEMDataTrain(data_test, mask_test)

dataloader = {
    'train': torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'val': torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
}


In [4]:
model = Unet(start_fm = 16)
model.cuda();

criterion = nn.CrossEntropyLoss()
# criterion = SoftDiceLoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

model = train(model,dataloader, criterion, learning_rate, optimizer, 100)

Epoch 0/99
----------
BEGIN TRAIN
train: criterion_loss: 0.748576, dice: 0.649276, iou: 0.762148
BEGIN EVAL
val: criterion_loss: 0.472612, dice: 0.542377, iou: 0.669645
saving best model
0m 23s
Epoch 1/99
----------
BEGIN TRAIN
train: criterion_loss: 0.340409, dice: 0.446281, iou: 0.584161
BEGIN EVAL
val: criterion_loss: 0.263678, dice: 0.369914, iou: 0.513143
saving best model
0m 24s
Epoch 2/99
----------
BEGIN TRAIN
train: criterion_loss: 0.216823, dice: 0.301450, iou: 0.442630
BEGIN EVAL
val: criterion_loss: 0.211755, dice: 0.282593, iou: 0.421296
saving best model
0m 24s
Epoch 3/99
----------
BEGIN TRAIN
train: criterion_loss: 0.170678, dice: 0.232537, iou: 0.362458
BEGIN EVAL
val: criterion_loss: 0.160154, dice: 0.217213, iou: 0.343038
saving best model
0m 24s
Epoch 4/99
----------
BEGIN TRAIN
train: criterion_loss: 0.145681, dice: 0.195966, iou: 0.315779
BEGIN EVAL
val: criterion_loss: 0.149388, dice: 0.197851, iou: 0.318201
saving best model
0m 24s
Epoch 5/99
----------
BEGIN TR

In [5]:
torch.save(model, ("Stuff/model_wts/Unet_16"))  